In [2]:
from manim import *
from numpy import linalg as npl
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline
from multiprocessing import Process

video_scene = " -v WARNING --progress_bar None --disable_caching mw_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s mw_Scene"

Manim Community v0.16.0.post0

Jupyter Capture Output v0.0.8


In [3]:
omega = 2
k_x = PI/3

def pulse_1_static(x):
    if (x < 6):
        return 4*np.sin(k_x * x) + 1
    else:
        return 1


def pulse_2(x, t):
    if (x > t*omega/k_x and x < t*omega/k_x + 6):
        return 4*np.sin(k_x * x - omega*t) + 1
    else:
        return 1

In [6]:
%%capture_video --path "animations/mechanic_wave/mechanic_wave_F1.mp4"
%%manim -qh --fps 60 $video_scene

class mw_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)
        time = ValueTracker(0)

        text1 = Tex(r"Horizontale Ausbreitung eines Wellenpulses mit $v_x$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(RIGHT)
        text2 = Tex(r"Vertikale Auslenkung ortsfester Punkte (1-4)", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(RIGHT)
        text2[0][35:41].set_color(RED)


        # Parameter
        omega = 2
        k_x = PI/3

        # Animationsparameter
        pulse_1_width = 1.5
        origin = [0, -0.25, 0]

        # Coordinate System
        x_range = [-0.2, 10, 1]
        y_range = [-4.5, 5.5, 1]

        x_length = 10
        y_length = 5.5

        npla = NumberPlane(
            x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}, ).move_to(origin)
        ax = Axes(x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, axis_config = {"stroke_width": 1, "stroke_opacity": 0.75}).move_to(origin)

        ax_xlabel = ax.get_x_axis_label(Tex(r"x$\,$(m)", font_size = 36)).shift(0.25*DOWN + 0.125*RIGHT)
        ax_ylabel = ax.get_y_axis_label(Tex(r"y$\,$(m)", font_size = 36)).shift(1.5*LEFT + 0.5*DOWN)

        coordinate_system = VGroup(npla, ax, ax_xlabel, ax_ylabel)


        # sine-Pulse 1
        sin_pulse_1 = ax.plot(lambda x: 4*np.sin(k_x * x) + 1, color = WHITE, stroke_width = pulse_1_width, x_range = [0, 0])
        line_pulse_1a = ax.plot(lambda x: 1, color = WHITE, stroke_width = pulse_1_width, x_range = [0, 10])
        line_pulse_1b = ax.plot(lambda x: 1, color = WHITE, stroke_width = pulse_1_width, x_range = [0, 0])

        def sin_updater_1(pulse):
            t = time.get_value() % (3*PI)
            pulse.become(ax.plot(lambda x: 4*np.sin(k_x * x - t*omega) + 1, color = WHITE, stroke_width = pulse_1_width, x_range = [max(0, t*omega/k_x - 6), min(t*omega/k_x, 10)]))#x_range = [max([0, t*omega]), min([5, t*omega])]))

        def line_1a_updater(line):
            t = time.get_value() % (3*PI)
            line.become(ax.plot(lambda x: 1, color = WHITE, stroke_width = pulse_1_width, x_range = [min(t*omega/k_x, 9.9999), 10]))

        def line_1b_updater(line):
            t = time.get_value() % (3*PI)
            line.become(ax.plot(lambda x: 1, color = WHITE, stroke_width = pulse_1_width, x_range = [0, max(0, min(t*omega/k_x - 6, 10))]))


        # sine-Pulse 2
        sin_pulse_2 = ax.plot(lambda x: 4*np.sin(k_x * x) + 1, color = RED, stroke_width = pulse_1_width, x_range = [0, 6])
        line_pulse_2a = ax.plot(lambda x: 1, color = RED, stroke_width = pulse_1_width, x_range = [6, 10])
        line_pulse_2b = ax.plot(lambda x: 1, color = RED, stroke_width = pulse_1_width, x_range = [0, 0])

        def sin_updater_2(pulse):
            t = time.get_value() % (3*PI)
            pulse.become(ax.plot(lambda x: 4*np.sin(k_x * x - t*omega) + 1, color = RED, stroke_width = pulse_1_width, x_range = [min(t*omega/k_x, 9.9999), min(t*omega/k_x + 6, 10)]))


        # static Dots for Pulse 1
        dot_x_positions = [0.5, 3, 4.5, 7]
        dot1_1 = Dot(ax.c2p(dot_x_positions[0], pulse_1_static(dot_x_positions[0])), radius = 0.075, color = WHITE)
        dot2_1 = Dot(ax.c2p(dot_x_positions[1], pulse_1_static(dot_x_positions[1])), radius = 0.075, color = WHITE)
        dot3_1 = Dot(ax.c2p(dot_x_positions[2], pulse_1_static(dot_x_positions[2])), radius = 0.075, color = WHITE)
        dot4_1 = Dot(ax.c2p(dot_x_positions[3], pulse_1_static(dot_x_positions[3])), radius = 0.075, color = WHITE)

        dot1_1_label = Text("1", color = RED, font_size = 24).move_to(ax.c2p(dot_x_positions[0], pulse_1_static(dot_x_positions[0]))).shift(0.3*LEFT)
        dot2_1_label = Text("2", color = RED, font_size = 24).move_to(ax.c2p(dot_x_positions[1], pulse_1_static(dot_x_positions[1]))).shift(0.25*LEFT + 0.25*DOWN)
        dot3_1_label = Text("3", color = RED, font_size = 24).move_to(ax.c2p(dot_x_positions[2], pulse_1_static(dot_x_positions[2]))).shift(0.35*DOWN)
        dot4_1_label = Text("4", color = RED, font_size = 24).move_to(ax.c2p(dot_x_positions[3], pulse_1_static(dot_x_positions[3]))).shift(0.35*DOWN)
    

        # Dot for Pulse 2
        dot1_2 = Dot(ax.c2p(dot_x_positions[0], pulse_2(dot_x_positions[0], 0)), radius = 0.075, color = RED)
        dot2_2 = Dot(ax.c2p(dot_x_positions[1], pulse_2(dot_x_positions[1], 0)), radius = 0.075, color = RED)
        dot3_2 = Dot(ax.c2p(dot_x_positions[2], pulse_2(dot_x_positions[2], 0)), radius = 0.075, color = RED)
        dot4_2 = Dot(ax.c2p(dot_x_positions[3], pulse_2(dot_x_positions[3], 0)), radius = 0.075, color = RED)

        dot1_2.number = 0
        dot2_2.number = 1
        dot3_2.number = 2
        dot4_2.number = 3

        def dot_updater(dot):
            t = time.get_value() % (3*PI)
            x_pos = dot_x_positions[dot.number]
            dot.become(Dot(ax.c2p(x_pos, pulse_2(x_pos, t)), radius = 0.075, color = RED))

        def line_2a_updater(line):
            t = time.get_value() % (3*PI)
            line.become(ax.plot(lambda x: 1, color = RED, stroke_width = pulse_1_width, x_range = [min(t*omega/k_x+6, 9.9999), 10]))

        def line_2b_updater(line):
            t = time.get_value() % (3*PI)
            line.become(ax.plot(lambda x: 1, color = RED, stroke_width = pulse_1_width, x_range = [0, min(t*omega/k_x, 10.00001)]))


        # Connectors
        cnct1 = DashedLine(ax.c2p(dot_x_positions[0], pulse_1_static(dot_x_positions[0])), ax.c2p(dot_x_positions[0], pulse_2(dot_x_positions[0], 1/20*PI)), color = WHITE, stroke_width = pulse_1_width).add_tip(tip_length = 0.15, tip_width = 0.15)
        cnct2 = DashedLine(ax.c2p(dot_x_positions[1], pulse_1_static(dot_x_positions[1])), ax.c2p(dot_x_positions[1], pulse_2(dot_x_positions[1], 1/20*PI)), color = WHITE, stroke_width = pulse_1_width).add_tip(tip_length = 0.15, tip_width = 0.15)
        cnct3 = DashedLine(ax.c2p(dot_x_positions[2], pulse_1_static(dot_x_positions[2])), ax.c2p(dot_x_positions[2], pulse_2(dot_x_positions[2], 1/20*PI)), color = WHITE, stroke_width = pulse_1_width).add_tip(tip_length = 0.15, tip_width = 0.15)
        cnct4 = DashedLine(ax.c2p(dot_x_positions[3], pulse_1_static(dot_x_positions[3])), ax.c2p(dot_x_positions[3], pulse_2(dot_x_positions[3], 1/20*PI)), color = WHITE, stroke_width = pulse_1_width).add_tip(tip_length = 0.15, tip_width = 0.15)

        cnct1.number = 0
        cnct2.number = 1
        cnct3.number = 2
        cnct4.number = 3

        def cnct_updater(cnct):
            t = time.get_value() % (3*PI)
            x_pos = dot_x_positions[cnct.number]
            cnct.become(DashedLine(ax.c2p(x_pos, pulse_1_static(x_pos)), ax.c2p(x_pos, pulse_2(x_pos, t)), color = WHITE, stroke_width = pulse_1_width).add_tip(tip_length = 0.15, tip_width = 0.15))


########## Animation ##########

        self.play(FadeIn(coordinate_system), run_time = 3)
        self.play(Write(text1), run_time = 3)
        self.play(FadeIn(sin_pulse_1), FadeIn(line_pulse_1a), FadeIn(line_pulse_1b), run_time = 3)

        sin_pulse_1.add_updater(sin_updater_1)
        line_pulse_1a.add_updater(line_1a_updater)
        line_pulse_1b.add_updater(line_1b_updater)

        self.wait(0.5)
        self.play(time.animate.set_value(4*PI), rate_func= linear, run_time = 12)
        self.wait(0.5)
        self.play(Unwrite(text1))
        self.wait(0.5)
        self.play(Write(text2), run_time = 3)
        self.wait(0.5)
        self.play(FadeIn(dot1_2), FadeIn(dot2_2), FadeIn(dot3_2), FadeIn(dot4_2), FadeIn(dot1_1_label), FadeIn(dot2_1_label), FadeIn(dot3_1_label), FadeIn(dot4_1_label), run_time = 1.5)
        self.wait(3)

        sin_pulse_1.remove_updater(sin_updater_1)
        line_pulse_1a.remove_updater(line_1a_updater)
        line_pulse_1b.remove_updater(line_1b_updater)

        self.add(dot1_1, dot2_1, dot3_1, dot4_1, sin_pulse_2, line_pulse_2a, line_pulse_2b)

        # # self.add(dot1_1_label, dot2_1_label, dot3_1_label, dot4_1_label)

        sin_pulse_2.add_updater(sin_updater_2)
        line_pulse_2a.add_updater(line_2a_updater)
        line_pulse_2b.add_updater(line_2b_updater)

        dot1_2.add_updater(dot_updater)
        dot2_2.add_updater(dot_updater)
        dot3_2.add_updater(dot_updater)
        dot4_2.add_updater(dot_updater)

        time = ValueTracker(0)
        
        self.play(time.animate.set_value(1/20*PI), rate_func= linear, run_time = 0.5)
        self.wait(0.5)
        self.play(Create(cnct1), Create(cnct2), Create(cnct3), Create(cnct4), run_time = 1.5)
        self.wait(3)

        cnct1.add_updater(cnct_updater)
        cnct2.add_updater(cnct_updater)
        cnct3.add_updater(cnct_updater)
        cnct4.add_updater(cnct_updater)

        self.play(time.animate.set_value(2*PI), rate_func= linear, run_time = 9)
        self.wait(2)


Output saved by overwring previous file at animations/mechanic_wave/mechanic_wave_F1.mp4.
